# TV Script Generation
In this project, you'll generate your own [Simpsons](https://en.wikipedia.org/wiki/The_Simpsons) TV scripts using RNNs.  You'll be using part of the [Simpsons dataset](https://www.kaggle.com/wcukierski/the-simpsons-by-the-data) of scripts from 27 seasons.  The Neural Network you'll build will generate a new TV script for a scene at [Moe's Tavern](https://simpsonswiki.com/wiki/Moe's_Tavern).
## Get the Data
The data is already provided for you.  You'll be using a subset of the original dataset.  It consists of only the scenes in Moe's Tavern.  This doesn't include other versions of the tavern, like "Moe's Cavern", "Flaming Moe's", "Uncle Moe's Family Feed-Bag", etc..

In [1]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper

data_dir = './data/simpsons/moes_tavern_lines.txt'
text = helper.load_data(data_dir)
# Ignore notice, since we don't use it for analysing the data
text = text[81:]

## Explore the Data
Play around with `view_sentence_range` to view different parts of the data.

In [2]:
view_sentence_range = (0, 10)

"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in text.split()})))
scenes = text.split('\n\n')
print('Number of scenes: {}'.format(len(scenes)))
sentence_count_scene = [scene.count('\n') for scene in scenes]
print('Average number of sentences in each scene: {}'.format(np.average(sentence_count_scene)))

sentences = [sentence for scene in scenes for sentence in scene.split('\n')]
print('Number of lines: {}'.format(len(sentences)))
word_count_sentence = [len(sentence.split()) for sentence in sentences]
print('Average number of words in each line: {}'.format(np.average(word_count_sentence)))

print()
print('The sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))

Dataset Stats
Roughly the number of unique words: 11492
Number of scenes: 262
Average number of sentences in each scene: 15.248091603053435
Number of lines: 4257
Average number of words in each line: 11.50434578341555

The sentences 0 to 10:
Moe_Szyslak: (INTO PHONE) Moe's Tavern. Where the elite meet to drink.
Bart_Simpson: Eh, yeah, hello, is Mike there? Last name, Rotch.
Moe_Szyslak: (INTO PHONE) Hold on, I'll check. (TO BARFLIES) Mike Rotch. Mike Rotch. Hey, has anybody seen Mike Rotch, lately?
Moe_Szyslak: (INTO PHONE) Listen you little puke. One of these days I'm gonna catch you, and I'm gonna carve my name on your back with an ice pick.
Moe_Szyslak: What's the matter Homer? You're not your normal effervescent self.
Homer_Simpson: I got my problems, Moe. Give me another one.
Moe_Szyslak: Homer, hey, you should not drink to forget your problems.
Barney_Gumble: Yeah, you should only drink to enhance your social skills.




## Implement Preprocessing Functions
The first thing to do to any dataset is preprocessing.  Implement the following preprocessing functions below:
- Lookup Table
- Tokenize Punctuation

### Lookup Table
To create a word embedding, you first need to transform the words to ids.  In this function, create two dictionaries:
- Dictionary to go from the words to an id, we'll call `vocab_to_int`
- Dictionary to go from the id to word, we'll call `int_to_vocab`

Return these dictionaries in the following tuple `(vocab_to_int, int_to_vocab)`

In [3]:
import numpy as np
import problem_unittests as tests
from collections import Counter


def create_lookup_tables(text):
    """
    Create lookup tables for vocabulary
    :param text: The text of tv scripts split into words
    :return: A tuple of dicts (vocab_to_int, int_to_vocab)
    """
    # TODO: Implement Function
    words_list = list(set(word for word in text))
    int_to_vocab={i:word for i, word in enumerate(words_list)}
    vocab_to_int= {word:i for i,word in enumerate(words_list)}
    
    return vocab_to_int,int_to_vocab


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_create_lookup_tables(create_lookup_tables)

Tests Passed


### Tokenize Punctuation
We'll be splitting the script into a word array using spaces as delimiters.  However, punctuations like periods and exclamation marks make it hard for the neural network to distinguish between the word "bye" and "bye!".

Implement the function `token_lookup` to return a dict that will be used to tokenize symbols like "!" into "||Exclamation_Mark||".  Create a dictionary for the following symbols where the symbol is the key and value is the token:
- Period ( . )
- Comma ( , )
- Quotation Mark ( " )
- Semicolon ( ; )
- Exclamation mark ( ! )
- Question mark ( ? )
- Left Parentheses ( ( )
- Right Parentheses ( ) )
- Dash ( -- )
- Return ( \n )

This dictionary will be used to token the symbols and add the delimiter (space) around it.  This separates the symbols as it's own word, making it easier for the neural network to predict on the next word. Make sure you don't use a token that could be confused as a word. Instead of using the token "dash", try using something like "||dash||".

In [4]:
def token_lookup():
    """
    Generate a dict to turn punctuation into a token.
    :return: Tokenize dictionary where the key is the punctuation and the value is the token
    """
    
    tokens={}
    tokens['.']='||Period||'
    tokens[',']='||Comma||'
    tokens['\"']='||QuotaionMark||'
    tokens[';']='||Semicolon||'
    tokens['!']='||ExclamationMark||'
    tokens['?']='||QuestionMark||'
    tokens['(']='||LeftParentheses||'
    tokens[')']='||RightParentheses||'
    tokens['--']='||Dash||'
    tokens['\n']='||Return||'
    
    
    # TODO: Implement Function
    return tokens

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_tokenize(token_lookup)

Tests Passed


## Preprocess all the data and save it
Running the code cell below will preprocess all the data and save it to file.

In [5]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Preprocess Training, Validation, and Testing Data
helper.preprocess_and_save_data(data_dir, token_lookup, create_lookup_tables)

# Check Point
This is your first checkpoint. If you ever decide to come back to this notebook or have to restart the notebook, you can start from here. The preprocessed data has been saved to disk.

In [5]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper
import numpy as np
import problem_unittests as tests

int_text, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()

## Build the Neural Network
You'll build the components necessary to build a RNN by implementing the following functions below:
- get_inputs
- get_init_cell
- get_embed
- build_rnn
- build_nn
- get_batches

### Check the Version of TensorFlow and Access to GPU

In [6]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from distutils.version import LooseVersion
import warnings
import tensorflow as tf

# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.0'), 'Please use TensorFlow version 1.0 or newer'
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.1.0-rc2
Default GPU Device: /gpu:0


### Input
Implement the `get_inputs()` function to create TF Placeholders for the Neural Network.  It should create the following placeholders:
- Input text placeholder named "input" using the [TF Placeholder](https://www.tensorflow.org/api_docs/python/tf/placeholder) `name` parameter.
- Targets placeholder
- Learning Rate placeholder

Return the placeholders in the following tuple `(Input, Targets, LearningRate)`

In [7]:
def get_inputs():
    """
    Create TF Placeholders for input, targets, and learning rate.
    :return: Tuple (input, targets, learning rate)
    """
    
    
    inputs = tf.placeholder(dtype=tf.int32,shape=(None,None),name="input")
    targets = tf.placeholder(dtype=tf.int32,shape=(None,None),name="targets")
    learning_rate = tf.placeholder(dtype=tf.float32,name="learning_rate")

    return inputs, targets, learning_rate
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_inputs(get_inputs)

Tests Passed


### Build RNN Cell and Initialize
Stack one or more [`BasicLSTMCells`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/BasicLSTMCell) in a [`MultiRNNCell`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/MultiRNNCell).
- The Rnn size should be set using `rnn_size`
- Initalize Cell State using the MultiRNNCell's [`zero_state()`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/MultiRNNCell#zero_state) function
    - Apply the name "initial_state" to the initial state using [`tf.identity()`](https://www.tensorflow.org/api_docs/python/tf/identity)

Return the cell and initial state in the following tuple `(Cell, InitialState)`

In [8]:
from tensorflow.contrib.rnn import MultiRNNCell
from tensorflow.contrib.rnn import BasicLSTMCell

def get_init_cell(batch_size, rnn_size):
    """
    Create an RNN Cell and initialize it.
    :param batch_size: Size of batches
    :param rnn_size: Size of RNNs
    :return: Tuple (cell, initialize state)
    """
    lstm_cell = BasicLSTMCell(rnn_size)
    multi_rnn_cell = MultiRNNCell([lstm_cell])
    
    init_state = tf.identity(multi_rnn_cell.zero_state(batch_size, tf.float32),name="initial_state")
    
    # TODO: Implement Function
    return multi_rnn_cell, init_state

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_init_cell(get_init_cell)

Tests Passed


### Word Embedding
Apply embedding to `input_data` using TensorFlow.  Return the embedded sequence.

In [9]:
import tensorflow as tf

def get_embed(input_data, vocab_size, embed_dim):
    """
    Create embedding for <input_data>.
    :param input_data: TF placeholder for text input.
    :param vocab_size: Number of words in vocabulary.
    :param embed_dim: Number of embedding dimensions
    :return: Embedded input.
    """
    
    embedding = tf.Variable(tf.random_uniform((vocab_size, embed_dim), -1, 1))
    embed = tf.nn.embedding_lookup(embedding, input_data)
    return embed


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_embed(get_embed)

(50, 5, 200)
Tests Passed


### Build RNN
You created a RNN Cell in the `get_init_cell()` function.  Time to use the cell to create a RNN.
- Build the RNN using the [`tf.nn.dynamic_rnn()`](https://www.tensorflow.org/api_docs/python/tf/nn/dynamic_rnn)
 - Apply the name "final_state" to the final state using [`tf.identity()`](https://www.tensorflow.org/api_docs/python/tf/identity)

Return the outputs and final_state state in the following tuple `(Outputs, FinalState)` 

In [10]:
def build_rnn(cell, inputs):
    """
    Create a RNN using a RNN Cell
    :param cell: RNN Cell
    :param inputs: Input text data
    :return: Tuple (Outputs, Final State)
    """
    # TODO: Implement Function
    outputs, state = tf.nn.dynamic_rnn(cell,inputs,dtype=tf.float32)
    
    final_state = tf.identity(state,"final_state")
    return outputs, final_state

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_build_rnn(build_rnn)

Tests Passed


### Build the Neural Network
Apply the functions you implemented above to:
- Apply embedding to `input_data` using your `get_embed(input_data, vocab_size, embed_dim)` function.
- Build RNN using `cell` and your `build_rnn(cell, inputs)` function.
- Apply a fully connected layer with a linear activation and `vocab_size` as the number of outputs.

Return the logits and final state in the following tuple (Logits, FinalState) 

In [11]:
def build_nn(cell, rnn_size, input_data, vocab_size, embed_dim):
    """
    Build part of the neural network
    :param cell: RNN cell
    :param rnn_size: Size of rnns
    :param input_data: Input data
    :param vocab_size: Vocabulary size
    :param embed_dim: Number of embedding dimensions
    :return: Tuple (Logits, FinalState)
    """
    # TODO: Implement Function'
    embedding = get_embed(input_data,vocab_size,embed_dim)
    outputs, final_state = build_rnn(cell,embedding)
    logits = tf.layers.dense(outputs,vocab_size)

    return logits, final_state

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_build_nn(build_nn)

Tests Passed


### Batches
Implement `get_batches` to create batches of input and targets using `int_text`.  The batches should be a Numpy array with the shape `(number of batches, 2, batch size, sequence length)`. Each batch contains two elements:
- The first element is a single batch of **input** with the shape `[batch size, sequence length]`
- The second element is a single batch of **targets** with the shape `[batch size, sequence length]`

If you can't fill the last batch with enough data, drop the last batch.

For exmple, `get_batches([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15], 2, 3)` would return a Numpy array of the following:
```
[
  # First Batch
  [
    # Batch of Input
    [[ 1  2  3], [ 7  8  9]],
    # Batch of targets
    [[ 2  3  4], [ 8  9 10]]
  ],
 
  # Second Batch
  [
    # Batch of Input
    [[ 4  5  6], [10 11 12]],
    # Batch of targets
    [[ 5  6  7], [11 12 13]]
  ]
]
```

### Batches
Implement `get_batches` to create batches of input and targets using `int_text`.  The batches should be a Numpy array with the shape `(number of batches, 2, batch size, sequence length)`. Each batch contains two elements:
- The first element is a single batch of **input** with the shape `[batch size, sequence length]`
- The second element is a single batch of **targets** with the shape `[batch size, sequence length]`

If you can't fill the last batch with enough data, drop the last batch.

For exmple, `get_batches([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15], 2, 3)` would return a Numpy array of the following:
```
[
  # First Batch
  [
    # Batch of Input
    [[ 1  2  3], [ 7  8  9]],
    # Batch of targets
    [[ 2  3  4], [ 8  9 10]]
  ],
 
  # Second Batch
  [
    # Batch of Input
    [[ 4  5  6], [10 11 12]],
    # Batch of targets
    [[ 5  6  7], [11 12 13]]
  ]
]
```

In [12]:
def get_batches(int_text, batch_size, seq_length):
    """
    Return batches of input and target
    :param int_text: Text with the words replaced by their ids
    :param batch_size: The size of batch
    :param seq_length: The length of sequence
    :return: Batches as a Numpy array
    """
    # TODO: Implement Function
    
    long_sequence_length = batch_size*seq_length
    long_sequence_count = len(int_text)//long_sequence_length
    long_sequence_size = long_sequence_length * long_sequence_count
    
    int_text_input = int_text[0:long_sequence_size]
    int_text_target = int_text[1:long_sequence_size+1]
    int_text_target[-1]= int_text_input[0]
    
    int_text_input = np.asarray(int_text_input,np.int32)
    int_text_target = np.asarray(int_text_target,np.int32)
    
    sequence_data = np.empty((long_sequence_count,2,batch_size,seq_length))
    
    index=0
    for x in range(long_sequence_count):
        x=x*long_sequence_length
        inner_input = np.reshape(int_text_input[x:x+long_sequence_length],(batch_size,seq_length))
        inner_target = np.reshape(int_text_target[x:x+long_sequence_length],(batch_size,seq_length))
        sequence_data[index,:,:,:]=[inner_input,inner_target]
        index+=1
    
    print(sequence_data[-1,1,-1,:])
        
    return sequence_data

tests.test_get_batches(get_batches)

[ 4476.  4477.  4478.  4479.     0.]
Tests Passed


## Neural Network Training
### Hyperparameters
Tune the following parameters:

- Set `num_epochs` to the number of epochs.
- Set `batch_size` to the batch size.
- Set `rnn_size` to the size of the RNNs.
- Set `embed_dim` to the size of the embedding.
- Set `seq_length` to the length of sequence.
- Set `learning_rate` to the learning rate.
- Set `show_every_n_batches` to the number of batches the neural network should print progress.

In [13]:
# Number of Epochs
num_epochs = 500
# Batch Size
batch_size =256
# RNN Size
rnn_size = 512
# Embedding Dimension Size
embed_dim = 200
# Sequence Length
seq_length = 12
# Learning Rate
learning_rate = 0.001
# Show stats for every n number of batches
show_every_n_batches = True

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
save_dir = './save'

### Build the Graph
Build the graph using the neural network you implemented.

In [14]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from tensorflow.contrib import seq2seq

train_graph = tf.Graph()
with train_graph.as_default():
    vocab_size = len(int_to_vocab)
    input_text, targets, lr = get_inputs()
    input_data_shape = tf.shape(input_text)

    cell, initial_state = get_init_cell(input_data_shape[0], rnn_size)
    logits, final_state = build_nn(cell, rnn_size, input_text, vocab_size, embed_dim)

    # Probabilities for generating words
    probs = tf.nn.softmax(logits, name='probs')

    # Loss function
    cost = seq2seq.sequence_loss(
        logits,
        targets,
        tf.ones([input_data_shape[0], input_data_shape[1]]))

    # Optimizer
    optimizer = tf.train.AdamOptimizer(lr)

    # Gradient Clipping
    gradients = optimizer.compute_gradients(cost)
    capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients]
    train_op = optimizer.apply_gradients(capped_gradients)

## Train
Train the neural network on the preprocessed data.  If you have a hard time getting a good loss, check the [forums](https://discussions.udacity.com/) to see if anyone is having the same problem.

In [15]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
batches = get_batches(int_text, batch_size, seq_length)

with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())

    for epoch_i in range(num_epochs):
        state = sess.run(initial_state, {input_text: batches[0][0]})

        for batch_i, (x, y) in enumerate(batches):
            feed = {
                input_text: x,
                targets: y,
                initial_state: state,
                lr: learning_rate}
            train_loss, state, _ = sess.run([cost, final_state, train_op], feed)

            # Show every <show_every_n_batches> batches
            if (epoch_i * len(batches) + batch_i) % show_every_n_batches == 0:
                print('Epoch {:>3} Batch {:>4}/{}   train_loss = {:.3f}'.format(
                    epoch_i,
                    batch_i,
                    len(batches),
                    train_loss))

    # Save Model
    saver = tf.train.Saver()
    saver.save(sess, save_dir)
    print('Model Trained and Saved')

[  956.  3746.  3831.  2354.  3831.  1687.  6175.   212.  3697.  2611.
  4175.  4540.]
Epoch   0 Batch    0/22   train_loss = 8.824
Epoch   0 Batch    1/22   train_loss = 8.789
Epoch   0 Batch    2/22   train_loss = 8.741
Epoch   0 Batch    3/22   train_loss = 8.682
Epoch   0 Batch    4/22   train_loss = 8.554
Epoch   0 Batch    5/22   train_loss = 8.311
Epoch   0 Batch    6/22   train_loss = 7.986
Epoch   0 Batch    7/22   train_loss = 7.552
Epoch   0 Batch    8/22   train_loss = 7.168
Epoch   0 Batch    9/22   train_loss = 6.941
Epoch   0 Batch   10/22   train_loss = 6.796
Epoch   0 Batch   11/22   train_loss = 6.829
Epoch   0 Batch   12/22   train_loss = 6.834
Epoch   0 Batch   13/22   train_loss = 6.703
Epoch   0 Batch   14/22   train_loss = 6.841
Epoch   0 Batch   15/22   train_loss = 6.801
Epoch   0 Batch   16/22   train_loss = 6.948
Epoch   0 Batch   17/22   train_loss = 7.016
Epoch   0 Batch   18/22   train_loss = 6.866
Epoch   0 Batch   19/22   train_loss = 6.713
Epoch   0 Bat

Epoch   8 Batch    7/22   train_loss = 4.826
Epoch   8 Batch    8/22   train_loss = 4.781
Epoch   8 Batch    9/22   train_loss = 4.864
Epoch   8 Batch   10/22   train_loss = 4.828
Epoch   8 Batch   11/22   train_loss = 5.018
Epoch   8 Batch   12/22   train_loss = 5.032
Epoch   8 Batch   13/22   train_loss = 4.914
Epoch   8 Batch   14/22   train_loss = 4.975
Epoch   8 Batch   15/22   train_loss = 4.998
Epoch   8 Batch   16/22   train_loss = 5.095
Epoch   8 Batch   17/22   train_loss = 5.117
Epoch   8 Batch   18/22   train_loss = 5.050
Epoch   8 Batch   19/22   train_loss = 4.922
Epoch   8 Batch   20/22   train_loss = 5.001
Epoch   8 Batch   21/22   train_loss = 4.901
Epoch   9 Batch    0/22   train_loss = 4.817
Epoch   9 Batch    1/22   train_loss = 4.846
Epoch   9 Batch    2/22   train_loss = 4.732
Epoch   9 Batch    3/22   train_loss = 4.860
Epoch   9 Batch    4/22   train_loss = 4.791
Epoch   9 Batch    5/22   train_loss = 4.844
Epoch   9 Batch    6/22   train_loss = 4.808
Epoch   9 

Epoch  16 Batch   14/22   train_loss = 4.244
Epoch  16 Batch   15/22   train_loss = 4.253
Epoch  16 Batch   16/22   train_loss = 4.320
Epoch  16 Batch   17/22   train_loss = 4.342
Epoch  16 Batch   18/22   train_loss = 4.294
Epoch  16 Batch   19/22   train_loss = 4.198
Epoch  16 Batch   20/22   train_loss = 4.231
Epoch  16 Batch   21/22   train_loss = 4.193
Epoch  17 Batch    0/22   train_loss = 4.098
Epoch  17 Batch    1/22   train_loss = 4.119
Epoch  17 Batch    2/22   train_loss = 4.038
Epoch  17 Batch    3/22   train_loss = 4.131
Epoch  17 Batch    4/22   train_loss = 4.093
Epoch  17 Batch    5/22   train_loss = 4.146
Epoch  17 Batch    6/22   train_loss = 4.093
Epoch  17 Batch    7/22   train_loss = 4.022
Epoch  17 Batch    8/22   train_loss = 3.985
Epoch  17 Batch    9/22   train_loss = 4.059
Epoch  17 Batch   10/22   train_loss = 4.036
Epoch  17 Batch   11/22   train_loss = 4.194
Epoch  17 Batch   12/22   train_loss = 4.186
Epoch  17 Batch   13/22   train_loss = 4.105
Epoch  17 

Epoch  24 Batch   21/22   train_loss = 3.654
Epoch  25 Batch    0/22   train_loss = 3.573
Epoch  25 Batch    1/22   train_loss = 3.551
Epoch  25 Batch    2/22   train_loss = 3.507
Epoch  25 Batch    3/22   train_loss = 3.608
Epoch  25 Batch    4/22   train_loss = 3.573
Epoch  25 Batch    5/22   train_loss = 3.619
Epoch  25 Batch    6/22   train_loss = 3.543
Epoch  25 Batch    7/22   train_loss = 3.503
Epoch  25 Batch    8/22   train_loss = 3.474
Epoch  25 Batch    9/22   train_loss = 3.552
Epoch  25 Batch   10/22   train_loss = 3.537
Epoch  25 Batch   11/22   train_loss = 3.653
Epoch  25 Batch   12/22   train_loss = 3.608
Epoch  25 Batch   13/22   train_loss = 3.541
Epoch  25 Batch   14/22   train_loss = 3.621
Epoch  25 Batch   15/22   train_loss = 3.642
Epoch  25 Batch   16/22   train_loss = 3.648
Epoch  25 Batch   17/22   train_loss = 3.683
Epoch  25 Batch   18/22   train_loss = 3.636
Epoch  25 Batch   19/22   train_loss = 3.616
Epoch  25 Batch   20/22   train_loss = 3.588
Epoch  25 

Epoch  33 Batch    6/22   train_loss = 3.049
Epoch  33 Batch    7/22   train_loss = 3.052
Epoch  33 Batch    8/22   train_loss = 3.007
Epoch  33 Batch    9/22   train_loss = 3.103
Epoch  33 Batch   10/22   train_loss = 3.099
Epoch  33 Batch   11/22   train_loss = 3.192
Epoch  33 Batch   12/22   train_loss = 3.130
Epoch  33 Batch   13/22   train_loss = 3.066
Epoch  33 Batch   14/22   train_loss = 3.132
Epoch  33 Batch   15/22   train_loss = 3.156
Epoch  33 Batch   16/22   train_loss = 3.094
Epoch  33 Batch   17/22   train_loss = 3.145
Epoch  33 Batch   18/22   train_loss = 3.097
Epoch  33 Batch   19/22   train_loss = 3.128
Epoch  33 Batch   20/22   train_loss = 3.048
Epoch  33 Batch   21/22   train_loss = 3.126
Epoch  34 Batch    0/22   train_loss = 3.063
Epoch  34 Batch    1/22   train_loss = 3.006
Epoch  34 Batch    2/22   train_loss = 3.003
Epoch  34 Batch    3/22   train_loss = 3.085
Epoch  34 Batch    4/22   train_loss = 3.056
Epoch  34 Batch    5/22   train_loss = 3.103
Epoch  34 

...

Epoch 465 Batch   18/22   train_loss = 0.266
Epoch 465 Batch   19/22   train_loss = 0.282
Epoch 465 Batch   20/22   train_loss = 0.250
Epoch 465 Batch   21/22   train_loss = 0.302
Epoch 466 Batch    0/22   train_loss = 0.272
Epoch 466 Batch    1/22   train_loss = 0.279
Epoch 466 Batch    2/22   train_loss = 0.294
Epoch 466 Batch    3/22   train_loss = 0.286
Epoch 466 Batch    4/22   train_loss = 0.280
Epoch 466 Batch    5/22   train_loss = 0.279
Epoch 466 Batch    6/22   train_loss = 0.286
Epoch 466 Batch    7/22   train_loss = 0.283
Epoch 466 Batch    8/22   train_loss = 0.273
Epoch 466 Batch    9/22   train_loss = 0.259
Epoch 466 Batch   10/22   train_loss = 0.269
Epoch 466 Batch   11/22   train_loss = 0.263
Epoch 466 Batch   12/22   train_loss = 0.262
Epoch 466 Batch   13/22   train_loss = 0.295
Epoch 466 Batch   14/22   train_loss = 0.260
Epoch 466 Batch   15/22   train_loss = 0.264
Epoch 466 Batch   16/22   train_loss = 0.293
Epoch 466 Batch   17/22   train_loss = 0.272
Epoch 466 

Epoch 474 Batch    3/22   train_loss = 0.286
Epoch 474 Batch    4/22   train_loss = 0.280
Epoch 474 Batch    5/22   train_loss = 0.279
Epoch 474 Batch    6/22   train_loss = 0.286
Epoch 474 Batch    7/22   train_loss = 0.283
Epoch 474 Batch    8/22   train_loss = 0.273
Epoch 474 Batch    9/22   train_loss = 0.259
Epoch 474 Batch   10/22   train_loss = 0.269
Epoch 474 Batch   11/22   train_loss = 0.262
Epoch 474 Batch   12/22   train_loss = 0.262
Epoch 474 Batch   13/22   train_loss = 0.295
Epoch 474 Batch   14/22   train_loss = 0.260
Epoch 474 Batch   15/22   train_loss = 0.263
Epoch 474 Batch   16/22   train_loss = 0.293
Epoch 474 Batch   17/22   train_loss = 0.272
Epoch 474 Batch   18/22   train_loss = 0.265
Epoch 474 Batch   19/22   train_loss = 0.282
Epoch 474 Batch   20/22   train_loss = 0.250
Epoch 474 Batch   21/22   train_loss = 0.301
Epoch 475 Batch    0/22   train_loss = 0.271
Epoch 475 Batch    1/22   train_loss = 0.278
Epoch 475 Batch    2/22   train_loss = 0.294
Epoch 475 

Epoch 482 Batch   10/22   train_loss = 0.268
Epoch 482 Batch   11/22   train_loss = 0.262
Epoch 482 Batch   12/22   train_loss = 0.262
Epoch 482 Batch   13/22   train_loss = 0.294
Epoch 482 Batch   14/22   train_loss = 0.260
Epoch 482 Batch   15/22   train_loss = 0.263
Epoch 482 Batch   16/22   train_loss = 0.293
Epoch 482 Batch   17/22   train_loss = 0.272
Epoch 482 Batch   18/22   train_loss = 0.265
Epoch 482 Batch   19/22   train_loss = 0.282
Epoch 482 Batch   20/22   train_loss = 0.250
Epoch 482 Batch   21/22   train_loss = 0.301
Epoch 483 Batch    0/22   train_loss = 0.271
Epoch 483 Batch    1/22   train_loss = 0.278
Epoch 483 Batch    2/22   train_loss = 0.293
Epoch 483 Batch    3/22   train_loss = 0.286
Epoch 483 Batch    4/22   train_loss = 0.280
Epoch 483 Batch    5/22   train_loss = 0.279
Epoch 483 Batch    6/22   train_loss = 0.286
Epoch 483 Batch    7/22   train_loss = 0.283
Epoch 483 Batch    8/22   train_loss = 0.272
Epoch 483 Batch    9/22   train_loss = 0.259
Epoch 483 

Epoch 490 Batch   17/22   train_loss = 0.272
Epoch 490 Batch   18/22   train_loss = 0.265
Epoch 490 Batch   19/22   train_loss = 0.282
Epoch 490 Batch   20/22   train_loss = 0.250
Epoch 490 Batch   21/22   train_loss = 0.301
Epoch 491 Batch    0/22   train_loss = 0.271
Epoch 491 Batch    1/22   train_loss = 0.278
Epoch 491 Batch    2/22   train_loss = 0.293
Epoch 491 Batch    3/22   train_loss = 0.286
Epoch 491 Batch    4/22   train_loss = 0.279
Epoch 491 Batch    5/22   train_loss = 0.278
Epoch 491 Batch    6/22   train_loss = 0.286
Epoch 491 Batch    7/22   train_loss = 0.283
Epoch 491 Batch    8/22   train_loss = 0.272
Epoch 491 Batch    9/22   train_loss = 0.259
Epoch 491 Batch   10/22   train_loss = 0.268
Epoch 491 Batch   11/22   train_loss = 0.262
Epoch 491 Batch   12/22   train_loss = 0.262
Epoch 491 Batch   13/22   train_loss = 0.294
Epoch 491 Batch   14/22   train_loss = 0.260
Epoch 491 Batch   15/22   train_loss = 0.263
Epoch 491 Batch   16/22   train_loss = 0.293
Epoch 491 

Epoch 499 Batch    2/22   train_loss = 0.293
Epoch 499 Batch    3/22   train_loss = 0.286
Epoch 499 Batch    4/22   train_loss = 0.279
Epoch 499 Batch    5/22   train_loss = 0.278
Epoch 499 Batch    6/22   train_loss = 0.286
Epoch 499 Batch    7/22   train_loss = 0.283
Epoch 499 Batch    8/22   train_loss = 0.272
Epoch 499 Batch    9/22   train_loss = 0.259
Epoch 499 Batch   10/22   train_loss = 0.268
Epoch 499 Batch   11/22   train_loss = 0.262
Epoch 499 Batch   12/22   train_loss = 0.261
Epoch 499 Batch   13/22   train_loss = 0.294
Epoch 499 Batch   14/22   train_loss = 0.260
Epoch 499 Batch   15/22   train_loss = 0.263
Epoch 499 Batch   16/22   train_loss = 0.293
Epoch 499 Batch   17/22   train_loss = 0.271
Epoch 499 Batch   18/22   train_loss = 0.265
Epoch 499 Batch   19/22   train_loss = 0.281
Epoch 499 Batch   20/22   train_loss = 0.249
Epoch 499 Batch   21/22   train_loss = 0.301
Model Trained and Saved


## Save Parameters
Save `seq_length` and `save_dir` for generating a new TV script.

In [16]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Save parameters for checkpoint
helper.save_params((seq_length, save_dir))

# Checkpoint

In [17]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import tensorflow as tf
import numpy as np
import helper
import problem_unittests as tests

_, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()
seq_length, load_dir = helper.load_params()

## Implement Generate Functions
### Get Tensors
Get tensors from `loaded_graph` using the function [`get_tensor_by_name()`](https://www.tensorflow.org/api_docs/python/tf/Graph#get_tensor_by_name).  Get the tensors using the following names:
- "input:0"
- "initial_state:0"
- "final_state:0"
- "probs:0"

Return the tensors in the following tuple `(InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)` 

In [18]:
def get_tensors(loaded_graph):
    """
    Get input, initial state, final state, and probabilities tensor from <loaded_graph>
    :param loaded_graph: TensorFlow graph loaded from file
    :return: Tuple (InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)
    """
    InputTensor = loaded_graph.get_tensor_by_name("input:0")
    InitialStateTensor = loaded_graph.get_tensor_by_name("initial_state:0")
    FinalStateTensor=loaded_graph.get_tensor_by_name("final_state:0")
    ProbsTensor = loaded_graph.get_tensor_by_name("probs:0")
    
    
    
    
    # TODO: Implement Function
    return InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_tensors(get_tensors)

Tests Passed


### Choose Word
Implement the `pick_word()` function to select the next word using `probabilities`.

In [19]:
def pick_word(probabilities, int_to_vocab):
    """
    Pick the next word in the generated text
    :param probabilities: Probabilites of the next word
    :param int_to_vocab: Dictionary of word ids as the keys and words as the values
    :return: String of the predicted word
    """
    # TODO: Implement Function
    i = np.argmax(probabilities)
    word = int_to_vocab[i]
    
    return word
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_pick_word(pick_word)

Tests Passed


## Generate TV Script
This will generate the TV script for you.  Set `gen_length` to the length of TV script you want to generate.

In [25]:
gen_length =500
# homer_simpson, moe_szyslak, or Barney_Gumble
prime_word = 'homer_simpson'

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(load_dir + '.meta')
    loader.restore(sess, load_dir)

    # Get Tensors from loaded model
    input_text, initial_state, final_state, probs = get_tensors(loaded_graph)

    # Sentences generation setup
    gen_sentences = [prime_word + ':']
    prev_state = sess.run(initial_state, {input_text: np.array([[96]])})

    # Generate sentences
    for n in range(gen_length):
        # Dynamic Input
        dyn_input = [[vocab_to_int[word] for word in gen_sentences[-seq_length:]]]
        dyn_seq_length = len(dyn_input[0])

        # Get Prediction
        probabilities, prev_state = sess.run(
            [probs, final_state],
            {input_text: dyn_input, initial_state: prev_state})
        
        
        #print(probabilities.shape)
        
        pred_word = pick_word(probabilities[0][dyn_seq_length-1], int_to_vocab)
        gen_sentences.append(pred_word)
    
    # Remove tokens
    tv_script = ' '.join(gen_sentences)
    for key, token in token_dict.items():
        ending = ' ' if key in ['\n', '(', '"'] else ''
        tv_script = tv_script.replace(' ' + token.lower(), key)
    tv_script = tv_script.replace('\n ', '\n')
    tv_script = tv_script.replace('( ', '(')
        
    print(tv_script)

INFO:tensorflow:Restoring parameters from ./save
homer_simpson:(upset) marge, you're making a complete fool of yourse.
barney_gumble: i can't be mad at you, homer.
homer_simpson:(looking at watch) outside, that's too...
homer_simpson:(excited) ah, i ain't.
carl_carlson:(calling) lenny, there are you, sweetheart. the other day i was a hundred miles outta here.
moe_szyslak: well, somebody's a grumpy gus, homer. just this place is a lotta talkin' about...(points to homer) where did i find that drop?...
moe_szyslak: hey, i've been using it backward.
marge_simpson:(talk-sings) the wordloaf festival?
homer_simpson: i had a feeling that says i saw marge, and i gotta tell you guys that just push buttons all day.
moe_szyslak: well, i gotta hand it to you what about the school?
moe_szyslak:(offended) sure pretty, but here's a compromise: and moe?
moe_szyslak: sure, but i ain't angry.
carl_carlson: and i'm magnanimous in victory.
barney_gumble: wow!


moe_szyslak: well, i gotta hand it to you han

# The TV Script is Nonsensical
It's ok if the TV script doesn't make any sense.  We trained on less than a megabyte of text.  In order to get good results, you'll have to use a smaller vocabulary or get more data.  Luckly there's more data!  As we mentioned in the begging of this project, this is a subset of [another dataset](https://www.kaggle.com/wcukierski/the-simpsons-by-the-data).  We didn't have you train on all the data, because that would take too long.  However, you are free to train your neural network on all the data.  After you complete the project, of course.
# Submitting This Project
When submitting this project, make sure to run all the cells before saving the notebook. Save the notebook file as "dlnd_tv_script_generation.ipynb" and save it as a HTML file under "File" -> "Download as". Include the "helper.py" and "problem_unittests.py" files in your submission.